In [1]:
import numpy as np
import pandas as pd
import scipy
import implicit
import os

# pd.options.display.max_rows=None
# pd.options.display.max_columns=None

# 경고 문구 off
import warnings
warnings.filterwarnings('ignore')

# 1. Project_Movielens 영화 추천

### (1) 데이터 준비와 전처리

In [2]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남김
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
ratings['ratings'].unique()

array([5, 3, 4])

In [5]:
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옴
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
movies['title'] = movies['title'].str.lower()
movies['genre'] = movies['genre'].str.lower()
movies.head()

,movie_id,title,genre
0,1,toy story (1995),animation|children's|comedy
1,2,jumanji (1995),adventure|children's|fantasy
2,3,grumpier old men (1995),comedy|romance
3,4,waiting to exhale (1995),comedy|drama
4,5,father of the bride part ii (1995),comedy


In [8]:
data = pd.merge(ratings, movies)
data.head()

,user_id,movie_id,ratings,timestamp,title,genre
0,1,1193,5,978300760,one flew over the cuckoo's nest (1975),drama
1,2,1193,5,978298413,one flew over the cuckoo's nest (1975),drama
2,12,1193,4,978220179,one flew over the cuckoo's nest (1975),drama
3,15,1193,4,978199279,one flew over the cuckoo's nest (1975),drama
4,17,1193,5,978158471,one flew over the cuckoo's nest (1975),drama


### (2) 데이터 탐색

In [9]:
# 유저 수
data['user_id'].nunique()

6039

In [10]:
# 영화 수
data['movie_id'].nunique()

3628

In [11]:
# 장르 수
data['genre'].nunique()

301

In [12]:
# 인기 많은 영화
movie_count = data.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

In [13]:
# 유저별 몇 개의 장르를 보고 있는지에 대한 통계
user_count = data.groupby('user_id')['genre'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: genre, dtype: float64

### (3) 내가 선호하는 영화 5가지를 추가

In [15]:
data.loc[data['title']=='matrix, the (1999)'].head(1)

,user_id,movie_id,ratings,timestamp,title,genre
122219,2,2571,4,978299773,"matrix, the (1999)",action|sci-fi|thriller


In [16]:
data.loc[data['title']=='back to the future (1985)'].head(1)

,user_id,movie_id,ratings,timestamp,title,genre
21777,1,1270,5,978300055,back to the future (1985),comedy|sci-fi


In [17]:
data.loc[data['title']=='jurassic park (1993)'].head(1)

,user_id,movie_id,ratings,timestamp,title,genre
101881,2,480,5,978299809,jurassic park (1993),action|adventure|sci-fi


In [18]:
data.loc[data['title']=='men in black (1997)'].head(1)

,user_id,movie_id,ratings,timestamp,title,genre
166207,3,1580,3,978297663,men in black (1997),action|adventure|comedy|sci-fi


In [19]:
data.loc[data['title']=='edward scissorhands (1990)'].head(1)

,user_id,movie_id,ratings,timestamp,title,genre
223334,5,2291,5,978244808,edward scissorhands (1990),drama|romance


In [20]:
# jw
my_favorite = pd.DataFrame([{'user_id':'jw', 'movie_id':2571, 'ratings':5, 'timestamp':978299773, 'title':'matrix, the (1999)', 'genre':'action|sci-fi|thriller'}, 
                            {'user_id':'jw', 'movie_id':1270, 'ratings':5, 'timestamp':978300055, 'title':'back to the future (1985)', 'genre':'comedy|sci-fi'},
                            {'user_id':'jw', 'movie_id':480, 'ratings':4, 'timestamp':978299809, 'title':'jurassic park (1993)', 'genre':'action|adventure|sci-fi'},
                            {'user_id':'jw', 'movie_id':1580, 'ratings':5, 'timestamp':978297663, 'title':'men in black (1997)', 'genre':'action|adventure|comedy|sci-fi'},
                            {'user_id':'jw', 'movie_id':2291, 'ratings':5, 'timestamp':978244808, 'title':'edward scissorhands (1990)', 'genre':'drama|romance'}])
my_favorite

,user_id,movie_id,ratings,timestamp,title,genre
0,jw,2571,5,978299773,"matrix, the (1999)",action|sci-fi|thriller
1,jw,1270,5,978300055,back to the future (1985),comedy|sci-fi
2,jw,480,4,978299809,jurassic park (1993),action|adventure|sci-fi
3,jw,1580,5,978297663,men in black (1997),action|adventure|comedy|sci-fi
4,jw,2291,5,978244808,edward scissorhands (1990),drama|romance


In [21]:
data = pd.concat([data, my_favorite])
data

,user_id,movie_id,ratings,timestamp,title,genre
0,1,1193,5,978300760,one flew over the cuckoo's nest (1975),drama
1,2,1193,5,978298413,one flew over the cuckoo's nest (1975),drama
2,12,1193,4,978220179,one flew over the cuckoo's nest (1975),drama
3,15,1193,4,978199279,one flew over the cuckoo's nest (1975),drama
4,17,1193,5,978158471,one flew over the cuckoo's nest (1975),drama
...,...,...,...,...,...,...
0,jw,2571,5,978299773,"matrix, the (1999)",action|sci-fi|thriller
1,jw,1270,5,978300055,back to the future (1985),comedy|sci-fi
2,jw,480,4,978299809,jurassic park (1993),action|adventure|sci-fi
3,jw,1580,5,978297663,men in black (1997),action|adventure|comedy|sci-fi


In [22]:
my_favorite['movie_id']

0    2571
1    1270
2     480
3    1580
4    2291
Name: movie_id, dtype: int64

In [23]:
my_favorite['ratings']

0    5
1    5
2    4
3    5
4    5
Name: ratings, dtype: int64

In [24]:
my_favorite['timestamp']

0    978299773
1    978300055
2    978299809
3    978297663
4    978244808
Name: timestamp, dtype: int64

In [25]:
my_favorite['title']

0            matrix, the (1999)
1     back to the future (1985)
2          jurassic park (1993)
3           men in black (1997)
4    edward scissorhands (1990)
Name: title, dtype: object

In [26]:
my_favorite['genre']

0            action|sci-fi|thriller
1                     comedy|sci-fi
2           action|adventure|sci-fi
3    action|adventure|comedy|sci-fi
4                     drama|romance
Name: genre, dtype: object

In [27]:
data.tail()

,user_id,movie_id,ratings,timestamp,title,genre
0,jw,2571,5,978299773,"matrix, the (1999)",action|sci-fi|thriller
1,jw,1270,5,978300055,back to the future (1985),comedy|sci-fi
2,jw,480,4,978299809,jurassic park (1993),action|adventure|sci-fi
3,jw,1580,5,978297663,men in black (1997),action|adventure|comedy|sci-fi
4,jw,2291,5,978244808,edward scissorhands (1990),drama|romance


### (4) 모델에 활용하기 위한 전처리

In [28]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [29]:
# 인덱싱이 잘 되었는지 확인
print(user_to_idx['jw'])
print(title_to_idx['matrix, the (1999)'])

6039
124


In [30]:
data.tail(10)

,user_id,movie_id,ratings,timestamp,title,genre
836473,5851,3607,5,957756608,one little indian (1973),comedy|drama|western
836474,5854,3026,4,958346883,slaughterhouse (1987),horror
836475,5854,690,3,957744257,"promise, the (versprechen, das) (1994)",romance
836476,5938,2909,4,957273353,"five wives, three secretaries and me (1998)",documentary
836477,5948,1360,5,1016563709,identification of a woman (identificazione di ...,drama
0,jw,2571,5,978299773,"matrix, the (1999)",action|sci-fi|thriller
1,jw,1270,5,978300055,back to the future (1985),comedy|sci-fi
2,jw,480,4,978299809,jurassic park (1993),action|adventure|sci-fi
3,jw,1580,5,978297663,men in black (1997),action|adventure|comedy|sci-fi
4,jw,2291,5,978244808,edward scissorhands (1990),drama|romance


In [31]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구하기
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체 
else:
    print('user_id column indexing Fail!!')

# title_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

user_id column indexing OK!!
title column indexing OK!!


In [32]:
# 인덱싱이 잘 되었는지 확인
print(user_to_idx['jw'])
print(title_to_idx['back to the future (1985)'])

6039
22


In [33]:
data.tail(10)

,user_id,movie_id,ratings,timestamp,title,genre
836473,1621,3607,5,957756608,3623,comedy|drama|western
836474,3481,3026,4,958346883,3624,horror
836475,3481,690,3,957744257,3625,romance
836476,4159,2909,4,957273353,3626,documentary
836477,1648,1360,5,1016563709,3627,drama
0,6039,2571,5,978299773,124,action|sci-fi|thriller
1,6039,1270,5,978300055,22,comedy|sci-fi
2,6039,480,4,978299809,107,action|adventure|sci-fi
3,6039,1580,5,978297663,175,action|adventure|comedy|sci-fi
4,6039,2291,5,978244808,250,drama|romance


In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 836483 entries, 0 to 4
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   user_id    836483 non-null  int64 
 1   movie_id   836483 non-null  int64 
 2   ratings    836483 non-null  int64 
 3   timestamp  836483 non-null  int64 
 4   title      836483 non-null  int64 
 5   genre      836483 non-null  object
dtypes: int64(5), object(1)
memory usage: 44.7+ MB


### (5) 분석

#### 1) ratings에 있는 유니크한 영화 개수

In [36]:
data.groupby('ratings')['movie_id'].unique()

ratings
3    [1193, 661, 914, 3408, 2355, 1197, 1287, 2804,...
4    [1193, 661, 914, 3408, 2355, 1197, 1287, 2804,...
5    [1193, 661, 914, 3408, 2355, 1197, 1287, 2804,...
Name: movie_id, dtype: object

#### 2) ratings에 있는 유니크한 사용자 수

In [37]:
data.groupby('ratings')['user_id'].unique()

ratings
3    [8, 11, 26, 44, 84, 89, 94, 96, 98, 112, 135, ...
4    [2, 3, 5, 12, 13, 14, 15, 19, 20, 21, 32, 33, ...
5    [0, 1, 4, 6, 7, 9, 10, 16, 17, 18, 22, 23, 24,...
Name: user_id, dtype: object

### (6) CSR matrix를 직접 만들어 보기

#### CSR(Compressed Sparse Row) Matrix

[1] https://gaussian37.github.io/math-la-sparse_matrix/

In [39]:
data.head()

,user_id,movie_id,ratings,timestamp,title,genre
0,0,1193,5,978300760,0,drama
1,1,1193,5,978298413,0,drama
2,2,1193,4,978220179,0,drama
3,3,1193,4,978199279,0,drama
4,4,1193,5,978158471,0,drama


In [40]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

df = np.array(data['ratings'])
row_indices = np.array(data['user_id'])
col_indices = np.array(data['movie_id'])
csr_matrix = csr_matrix((df, (row_indices, col_indices)), dtype=int)
print(csr_matrix)

  (0, 1)	5
  (0, 48)	5
  (0, 150)	5
  (0, 260)	4
  (0, 527)	5
  (0, 531)	4
  (0, 588)	4
  (0, 594)	4
  (0, 595)	5
  (0, 608)	4
  (0, 661)	3
  (0, 720)	3
  (0, 745)	3
  (0, 783)	4
  (0, 914)	3
  (0, 919)	4
  (0, 938)	4
  (0, 1022)	5
  (0, 1028)	5
  (0, 1029)	5
  (0, 1035)	5
  (0, 1097)	4
  (0, 1193)	5
  (0, 1197)	3
  (0, 1207)	4
  :	:
  (6036, 3280)	5
  (6036, 3939)	5
  (6037, 366)	5
  (6037, 553)	4
  (6037, 1219)	5
  (6037, 1261)	5
  (6037, 1321)	5
  (6037, 1333)	3
  (6037, 1347)	5
  (6037, 1407)	4
  (6037, 1924)	3
  (6037, 1973)	5
  (6037, 1994)	5
  (6037, 2455)	4
  (6037, 2710)	5
  (6037, 2867)	3
  (6037, 3101)	4
  (6037, 3652)	3
  (6037, 3917)	5
  (6038, 2521)	3
  (6039, 480)	4
  (6039, 1270)	5
  (6039, 1580)	5
  (6039, 2291)	5
  (6039, 2571)	5


### (7) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련

In [41]:
from implicit.als import AlternatingLeastSquares

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [42]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [43]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줌)
csr_matrix_transpose = csr_matrix.T
csr_matrix_transpose

<3953x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [44]:
# 모델 훈련
als_model.fit(csr_matrix_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [45]:
data

,user_id,movie_id,ratings,timestamp,title,genre
0,0,1193,5,978300760,0,drama
1,1,1193,5,978298413,0,drama
2,2,1193,4,978220179,0,drama
3,3,1193,4,978199279,0,drama
4,4,1193,5,978158471,0,drama
...,...,...,...,...,...,...
0,6039,2571,5,978299773,124,action|sci-fi|thriller
1,6039,1270,5,978300055,22,comedy|sci-fi
2,6039,480,4,978299809,107,action|adventure|sci-fi
3,6039,1580,5,978297663,175,action|adventure|comedy|sci-fi


In [46]:
jw, back_to_the_future = user_to_idx['jw'], title_to_idx['back to the future (1985)']
jw_vector, back_to_the_future_vector = als_model.user_factors[jw], als_model.item_factors[back_to_the_future]

In [47]:
jw_vector

array([ 0.5601568 ,  0.12830903, -0.8325358 ,  0.8392417 ,  0.2022972 ,
        0.69815475,  0.34317902, -0.42678243,  0.36359808, -0.10233273,
        0.6514361 , -0.9706544 , -0.40327004,  0.10845129, -0.59275484,
       -1.1252545 ,  0.60626763, -0.27778882, -0.39421654, -0.32602358,
        0.2757489 ,  0.3461155 , -0.2129025 ,  0.2450919 ,  0.3413653 ,
       -0.6737792 ,  0.70638233,  0.21828377,  0.26244882,  0.445891  ,
       -0.05840042, -0.16608115,  0.46485952,  0.42298305, -1.0642406 ,
        0.55791837, -0.4875727 ,  0.4738991 ,  0.5734972 , -0.1958104 ,
       -0.08019236,  0.512016  ,  0.76761895, -0.30769342, -1.0163072 ,
       -0.50192654,  0.18982808,  0.5288922 ,  0.1546551 ,  0.24734817,
       -0.2382417 ,  0.4110294 , -0.8127139 ,  0.19511954,  0.54626364,
       -0.25551394, -0.18541437, -0.3175681 , -0.10459267, -0.56071764,
        0.64708185,  0.02488183,  0.61636126,  0.2767295 , -0.18990722,
       -0.30020407,  0.833412  ,  0.13431638,  0.5546513 ,  0.68

In [48]:
back_to_the_future_vector

array([ 2.17969082e-02,  1.16396425e-02,  1.33980736e-02, -1.58982500e-02,
       -1.44099919e-02,  1.43877405e-03,  1.15533080e-02,  6.14921749e-03,
        7.58288382e-03,  2.12253761e-02,  2.59138551e-02,  1.04727549e-02,
        1.45958196e-02,  7.97984470e-03,  1.78581476e-02,  1.36655718e-02,
        3.97562701e-03, -3.39954253e-03,  1.55738425e-02,  1.12086358e-02,
        1.00537408e-02,  1.91313736e-02,  2.40830146e-02,  7.52766442e-04,
       -2.14867238e-02,  1.12657193e-02, -4.85632103e-03,  1.43237098e-03,
       -9.29659046e-03,  1.38987396e-02,  5.59813529e-03, -6.88669970e-03,
        1.27616916e-02,  1.42329019e-02,  3.25341127e-03,  1.23433406e-02,
        1.64114852e-02,  2.96956254e-03,  1.10593752e-03,  1.30685652e-03,
        1.87773071e-02,  9.21714213e-03,  6.47729356e-03, -4.60521042e-05,
        9.78586823e-03,  1.77996960e-02,  7.71925738e-03,  5.01486100e-03,
       -1.81384329e-02, -2.91121029e-03, -1.87291838e-02,  7.27904914e-03,
        1.46686137e-02,  

In [49]:
# jw과 back_to_the_future를 내적하는 코드
np.dot(jw_vector, back_to_the_future_vector)

-0.057947073

In [50]:
edward_scissorhands = title_to_idx['edward scissorhands (1990)']
edward_scissorhands_vector = als_model.item_factors[edward_scissorhands]
np.dot(jw_vector, edward_scissorhands_vector)

-0.0026252908

### (8) 내가 좋아하는 영화와 비슷한 영화 찾기

In [51]:
favorite_movie = 'men in black (1997)'
movie_id = title_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(175, 1.0000001),
 (766, 0.7382495),
 (147, 0.72693783),
 (1611, 0.717476),
 (851, 0.7107691),
 (159, 0.68756413),
 (2271, 0.68541074),
 (2165, 0.68107915),
 (1051, 0.67360014),
 (1563, 0.6603932),
 (269, 0.6599378),
 (2305, 0.6522188),
 (1897, 0.6365901),
 (1120, 0.62983066),
 (574, 0.62916505)]

In [52]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_movie]

['men in black (1997)',
 'goldeneye (1995)',
 'under siege (1992)',
 'halloween ii (1981)',
 'iron giant, the (1999)',
 'manhattan (1979)',
 'trial, the (le procès) (1963)',
 "mary shelley's frankenstein (1994)",
 'mallrats (1995)',
 'love letter, the (1999)',
 'goodfellas (1990)',
 'farinelli: il castrato (1994)',
 'end of violence, the (1997)',
 'jaws 2 (1978)',
 'jack (1996)']

In [53]:
# 몇 번 더 반복해서 확인하기 위해 위의 코드를 함수로 만들고 확인
def get_similar_movie(movie_title: str):
    movie_id = title_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_title[i[0]] for i in similar_movie]
    return similar_movie

In [54]:
get_similar_movie('titanic (1997)')

['titanic (1997)',
 'forever young (1992)',
 'crazy in alabama (1999)',
 'absolute power (1997)',
 'friday the 13th: the final chapter (1984)',
 'con air (1997)',
 'angels and insects (1995)',
 'honey, i shrunk the kids (1989)',
 'star trek vi: the undiscovered country (1991)',
 'richard iii (1995)']

In [55]:
get_similar_movie('wizard of oz, the (1939)')

['wizard of oz, the (1939)',
 'snake eyes (1998)',
 'titanic (1953)',
 'frankenstein (1931)',
 'pleasantville (1998)',
 'fantasia (1940)',
 'bad seed, the (1956)',
 'american werewolf in london, an (1981)',
 'lady and the tramp (1955)',
 'fear of a black hat (1993)']

### (9) 내가 가장 좋아할 만한 영화들을 추천받기

In [56]:
user = user_to_idx['jw']

movie_recommended = als_model.recommend(user, csr_matrix, N=20, filter_already_liked_items=True)
movie_recommended

[(589, 0.6567018),
 (110, 0.38774735),
 (2916, 0.3718121),
 (2791, 0.3285446),
 (1097, 0.2995274),
 (260, 0.2932653),
 (1673, 0.29015797),
 (1544, 0.27253756),
 (780, 0.2660862),
 (1259, 0.26543367),
 (457, 0.2622699),
 (1527, 0.2561697),
 (1965, 0.2556075),
 (3175, 0.24485397),
 (32, 0.24358366),
 (1196, 0.24313454),
 (2455, 0.2396796),
 (2657, 0.23488611),
 (2716, 0.22245176),
 (1240, 0.2166595)]

In [57]:
[idx_to_title[i[0]] for i in movie_recommended]

['meet me in st. louis (1944)',
 'groundhog day (1993)',
 'omega code, the (1999)',
 'blood beach (1981)',
 'brazil (1985)',
 'living in oblivion (1995)',
 'enchanted april (1991)',
 'sinbad and the eye of the tiger (1977)',
 'crimson pirate, the (1952)',
 'sliver (1993)',
 'nell (1994)',
 'farewell to arms, a (1932)',
 'teenage mutant ninja turtles ii: the secret of the ooze (1991)',
 'twice upon a yesterday (1998)',
 'hercules (1997)',
 'net, the (1995)',
 'bad taste (1987)',
 'vampire in brooklyn (1995)',
 'my giant (1998)',
 'when a man loves a woman (1994)']

In [58]:
# 모델이 'meet me in st. louis (1944)'를 추천한 이유
meet_me_in_st_louis = title_to_idx['meet me in st. louis (1944)']
explain = als_model.explain(user, csr_matrix, itemid=meet_me_in_st_louis)
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[("soldier's daughter never cries, a (1998)", 0.2462543206992176),
 ('grand day out, a (1992)', 0.1695729356816319),
 ('paper, the (1994)', 0.14051798941778365),
 ('dying young (1991)', 0.06410382154109331),
 ("white men can't jump (1992)", 0.02600450367165514)]

# 2. References

[1] https://gaussian37.github.io/math-la-sparse_matrix/

[2] https://lms.aiffel.io/course/382/node/498